<a href="https://colab.research.google.com/github/starryesh22/Google_Colab/blob/main/0607_Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import wfdb
import numpy as np
import pandas as pd
from glob import glob
import argparse
import os


'''

1) wfdb 라이브러리는 생체 신호 및 시계열 데이터의 읽기, 쓰기, 처리를 위한 도구를 제공
이는 생체 의학 신호 처리 분야에서 주로 사용

2) numpy는 대규모 다차원 배열 및 행렬 지원과 이러한 배열에 대해 작동하는 수학 함수 컬렉션을 제공하는 
데이터 분석 및 조작에 널리 사용되는 라이브러리 

3) pandas는 데이터 조작 및 분석을 위한 데이터 구조 및 도구를 제공하는 라이브러리로,
 데이터 프레임을 포함.

4) glob 모듈은 Unix 쉘에서 사용되는 규칙에 따라 지정된 패턴과 일치하는 
모든 경로명을 찾는 glob() 함수를 제공. 결과는 임의의 순서로 반환됨

5)  argparse 모듈은 Python에서 명령줄 인수를 처리하는 방법을 제공
명령줄 인수 및 옵션을 구문 분석하는 과정을 단순화하여 사용자 친화적인 
명령줄 인터페이스 작성을 용이하게 함.

6) os 모듈은 플랫폼에 독립적인 방식으로 운영 체제와 상호 작용하는 기능을 제공. 
파일 및 디렉토리 생성, 삭제, 수정 등 운영 체제 관련 작업에 대한 다양한 함수를 제공
이러한 임포트는 데이터 분석 및 신호 처리 작업에서 흔히 사용되며, 
생체 신호 데이터를 처리하고 배열 및 데이터 프레임을 조작하며, 명령줄 인수를 처리하고 
운영 체제와 상호 작용하는 데 도움이 됨.

'''

In [ ]:
#1. gen_reference_csv 함수

def gen_reference_csv(data_dir, reference_csv):
    if not os.path.exists(reference_csv):
        recordpaths = glob(os.path.join(data_dir, '*.hea'))
        results = []
        for recordpath in recordpaths:
            patient_id = recordpath.split('/')[-1][:-4]
            _, meta_data = wfdb.rdsamp(recordpath[:-4])
            sample_rate = meta_data['fs']
            signal_len = meta_data['sig_len']
            age = meta_data['comments'][0]
            sex = meta_data['comments'][1]
            dx = meta_data['comments'][2]
            age = age[5:] if age.startswith('Age: ') else np.NaN
            sex = sex[5:] if sex.startswith('Sex: ') else 'Unknown'
            dx = dx[4:] if dx.startswith('Dx: ') else ''
            results.append([patient_id, sample_rate, signal_len, age, sex, dx])
        df = pd.DataFrame(data=results, columns=['patient_id', 'sample_rate', 'signal_len', 'age', 'sex', 'dx'])
        df.sort_values('patient_id').to_csv(reference_csv, index=None)

'''  

코드는 gen_reference_csv라는 함수로 정의되어 있음.
데이터 디렉토리와 참조 CSV 파일을 입력으로 받음

- reference_csv 파일이 존재하지 않는 경우에만 아래의 작업을 수행함
- data_dir에서 '*.hea' 확장자를 가진 파일들의 경로를 찾아서 recordpaths에 저장
- 빈 리스트 results를 생성
- recordpaths에 있는 각 파일에 대해 반복문을 실행
- recordpath에서 확장자를 제외한 파일 이름을 추출하여 patient_id 변수에 저장
- wfdb.rdsamp() 함수를 사용하여 해당 레코드 파일의 메타 데이터를 읽어옴
- 메타 데이터의 샘플 레이트(sample_rate), 신호 길이(signal_len), 나이(age), 성별(sex), 진단(dx)정보 추출
- 나이, 성별, 진단 정보가 'Age: ', 'Sex: ', 'Dx: '로 시작하는 경우 해당 부분을 제외한 값을 저장
-  그렇지 않은 경우에는 각각 NaN, 'Unknown', 빈 문자열을 저장
- results 리스트에 [patient_id, sample_rate, signal_len, age, sex, dx] 형태로 정보를 추가함.
- results를 데이터 프레임으로 변환하여 df에 저장합니다. 
  열 이름은 'patient_id', 'sample_rate', 'signal_len', 'age', 'sex', 'dx'로 지정
- 'patient_id' 열을 기준으로 데이터 프레임을 정렬하고, 인덱스 없이 reference_csv에 저장
- 이 함수는 주어진 데이터 디렉토리에 있는 레코드 파일들의 메타 데이터를 읽어와서 참조 CSV 파일에 저장.

'''

In [ ]:
# 2. gen_label_csv  함수

def gen_label_csv(label_csv, reference_csv, dx_dict, classes):
    if not os.path.exists(label_csv):
        results = []
        df_reference = pd.read_csv(reference_csv)
        for _, row in df_reference.iterrows():
            patient_id = row['patient_id']
            dxs = [dx_dict.get(code, '') for code in row['dx'].split(',')]
            labels = [0] * 9
            for idx, label in enumerate(classes):
                if label in dxs:
                    labels[idx] = 1
            results.append([patient_id] + labels)
        df = pd.DataFrame(data=results, columns=['patient_id'] + classes)
        n = len(df)
        folds = np.zeros(n, dtype=np.int8)
        for i in range(10):
            start = int(n * i / 10)
            end = int(n * (i + 1) / 10)
            folds[start:end] = i + 1
        df['fold'] = np.random.permutation(folds)
        columns = df.columns
        df['keep'] = df[classes].sum(axis=1)
        df = df[df['keep'] > 0]
        df[columns].to_csv(label_csv, index=None)


'''

gen_label_csv라는 함수로 정의
함수는 레이블 CSV 파일, 참조 CSV 파일, 진단 코드 사전(dx_dict), 그리고 클래스(classes)를 입력으로 받음
label_csv 파일이 존재하지 않는 경우에만 아래의 작업을 수행
빈 리스트 results를 생성
reference_csv 파일을 읽어와서 df_reference 데이터프레임에 저장
df_reference의 각 행에 대해 반복문을 실행
각 행에서 'patient_id'와 'dx' 정보를 추출
'dx' 정보를 쉼표로 구분하여 리스트로 분할한 후, 각 코드에 대응하는 값을 dx_dict에서 찾아서 dxs 리스트에 저장
길이가 9인 0으로 초기화된 labels 리스트를 생성
classes에 있는 각 레이블에 대해 반복문을 실행
만약 해당 레이블이 dxs에 포함되어 있다면, labels 리스트에서 해당 인덱스를 1로 변경
[patient_id] + labels 형태로 정보를 results 리스트에 추가
results를 데이터프레임으로 변환하여 df에 저장합니다. 열 이름은 'patient_id'와 classes로 지정
데이터프레임의 행 수를 n에 저장
길이가 n이고 모든 요소가 0인 배열 folds를 생성
10번 반복문을 실행하여 각 반복에서 folds의 일부 구간을 해당 반복 번호로 설정
df에 'fold' 열을 추가하고, folds 배열을 무작위로 섞어 값을 할당.
columns에 df의 열 이름을 저장
df의 classes 열을 합하여 'keep' 열에 저장합
'keep' 열의 값이 0보다 큰 행만 선택하여 df를 필터링
df의 'patient_id'와 columns 열만을 가지고 label_csv에 저장
이 함수는 주어진 참조 CSV 파일을 기반으로 레이블 CSV 파일을 생성. 
레이블은 진단 코드에 따라 클래스로 지정, 데이터프레임에는 'patient_id', 클래스 열, 'fold', 'keep' 열이 포함

'''

In [ ]:
# 2.  __main__ 블록 정의

if __name__ == "__main__":
    leads = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
    dx_dict = {
        '426783006': 'SNR', # Normal sinus rhythm
        '164889003': 'AF', # Atrial fibrillation
        '270492004': 'IAVB', # First-degree atrioventricular block
        '164909002': 'LBBB', # Left bundle branch block
        '713427006': 'RBBB', # Complete right bundle branch block
        '59118001': 'RBBB', # Right bundle branch block
        '284470004': 'PAC', # Premature atrial contraction
        '63593006': 'PAC', # Supraventricular premature beats
        '164884008': 'PVC', # Ventricular ectopics
        '429622005': 'STD', # ST-segment depression
        '164931005': 'STE', # ST-segment elevation
    }
    classes = ['SNR', 'AF', 'IAVB', 'LBBB', 'RBBB', 'PAC', 'PVC', 'STD', 'STE']
    parser = argparse.ArgumentParser()
    parser.add_argument('--data-dir', type=str, default='data/CPSC', help='Directory to dataset')
    args = parser.parse_args()
    data_dir = args.data_dir
    reference_csv = os.path.join(data_dir, 'reference.csv')
    label_csv = os.path.join(data_dir, 'labels.csv')
    gen_reference_csv(data_dir, reference_csv)
    gen_label_csv(label_csv, reference_csv, dx_dict, classes)

''' 

 __main__ 블록으로 구성 (__main__ 블록은 스크립트가 직접 실행될 때 실행되는 부분)
이 블록에서는 몇 가지 변수들을 설정한 후, gen_reference_csv와 gen_label_csv 함수를 호출하여
참조 CSV 파일과 레이블 CSV 파일을 생성.

leads는 사용할 리드(심전도의 리드)들을 나타내는 문자열 리스트임
dx_dict는 진단 코드와 해당하는 레이블을 매핑한 사전(dictionary)
classes는 사용할 클래스(진단)들을 나타내는 문자열 리스트임
argparse 모듈을 사용하여 명령줄 인수를 처리하기 위한 parser 객체를 생성

--data-dir 인수를 추가하여 데이터셋 디렉토리를 지정함. 기본값은 'data/CPSC'임.

args에는 parser.parse_args()를 통해 파싱한 인수가 저장됨
data_dir 변수에는 args.data_dir 값을 저장
reference_csv 변수에는 데이터 디렉토리와 'reference.csv' 파일 이름을 합쳐서 저장
label_csv 변수에는 데이터 디렉토리와 'labels.csv' 파일 이름을 합쳐서 저장
gen_reference_csv 함수를 호출하여 참조 CSV 파일을 생성
gen_label_csv 함수를 호출하여 레이블 CSV 파일을 생성
__main__ 블록은 해당 스크립트를 직접 실행할 때만 실행되며, 
주어진 데이터 디렉토리에서 참조 CSV 파일과 레이블 CSV 파일을 생성

'''    